In [3]:
import mdtraj
import numpy as np
import os
import alphaspace2 as al
from alphaspace2.Community import genCommunityPocket
import alphaspace2.Features as features


Applying community detection for full CDK2 surface

In [5]:
this_dir = os.path.abspath('')
cdk2_prot = mdtraj.load(this_dir + '/DataSet/CDK2_Communities/cdk2_prot.pdb')
al.annotateVinaAtomTypes(pdbqt=this_dir + "/DataSet/CDK2_Communities/cdk2_prot.pdbqt", receptor=cdk2_prot)

In [6]:
ss_cdk2 = al.Snapshot()
ss_cdk2.run(cdk2_prot)

In [7]:
temp_space_dict = {}
temp_coords_dict = {}
temp_contact_dict = {}
temp_score_dict = {}
temp_centroids_dict = {}
for px,pocket in enumerate(ss_cdk2.pockets):
    temp_coords_dict[px] = [list(b.xyz) for b in pocket.betas]
    temp_score_dict[px] = np.array([min(b.scores) for b in  pocket.betas])
    temp_space_dict[px] = np.array([b.space for b in  pocket.betas])
    temp_contact_dict[px] = list(pocket.lining_atoms_idx)
    temp_centroids_dict[px] = pocket.centroid
prot_coords = cdk2_prot.xyz[0]*10   ### change nm to Angstrom
#generate communities
surface_communities = genCommunityPocket(prot_coords, temp_centroids_dict, temp_space_dict,
                                            temp_contact_dict, temp_score_dict,
                                            corecut = 100, auxcut = 30, tight_option = True, tight_cutoff_core = 12.5, tight_cutoff_aux = 6.5)


Calculate more features of beta clusters community

In [10]:
surface_communities_props = {}
for cx,community in surface_communities.items():
    temp_coords_array = []
    for pock in community['core_pockets'] + community['aux_pockets']:
        temp_coords_array.extend(temp_coords_dict[pock])
    temp_coords_array = np.array(temp_coords_array)
    volume = features._get_grid_volume(temp_coords_array)
    occluded_asa = features._get_pharmacophore_fingerprint(cdk2_prot,temp_coords_array)
    surface_communities_props[cx] = {}
    surface_communities_props[cx]['space'] = community['space']
    surface_communities_props[cx]['score'] = community['score']
    surface_communities_props[cx]['volume'] = volume
    surface_communities_props[cx]['occluded_asa'] = occluded_asa['Total_OASA']    
    

In [24]:
print("Community\t{}\t{}\t{}\t{}".format('Space', 'Score', 'Volume', 'Occluded_ASA'))
for cx,props in surface_communities_props.items():
    print("{}\t\t{}\t{}\t{}\t{}".format(cx,round(props['space']),round(props['score']),round(props['volume']),round(props['occluded_asa'])))
    

Community	Space	Score	Volume	Occluded_ASA
0		1472.0	-29.0	690.0	689.0
1		1199.0	-19.0	574.0	740.0
2		889.0	-16.0	474.0	563.0
3		885.0	-20.0	565.0	609.0
4		659.0	-14.0	414.0	538.0
5		647.0	-16.0	393.0	418.0
6		457.0	-10.0	317.0	424.0
7		406.0	-10.0	200.0	277.0
8		351.0	-5.0	168.0	214.0
9		335.0	-7.0	187.0	254.0
10		285.0	-6.0	141.0	228.0
11		283.0	-7.0	262.0	289.0
12		260.0	-7.0	184.0	254.0
13		173.0	-6.0	166.0	168.0
14		131.0	-4.0	92.0	105.0
15		111.0	-2.0	57.0	113.0
